Define all the import necessary for the assignment

In [ ]:
%matplotlib notebook

import jupyros as jr
import rospy
import time
import actionlib
import actionlib.msg
import assignment_2_2022.msg
import assignment_2_2022
import sys
import select
import math
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt

from std_srvs.srv import *
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Point, Pose, Twist
from assignment_2_2022.msg import Position_vel
from sensor_msgs.msg import LaserScan
from matplotlib.animation import FuncAnimation

Define the value of the variable cancel_goal & goal_service

In [ ]:
cancel_goal = False
goal_service = None

Now defines a function `publishVal` that extracts position and velocity information from `msg` and pubblish a custom message.

In [ ]:
def publishVal(msg):
    # recall the global publisher as pub
    global pub
    
    # initialize the postion and the velocity from the message
    pos = msg.pose.pose.position
    velocity = msg.twist.twist.linear

    # Initialize a custom message
    pos_velox = Position_vel()

    # assign the parameters of the custom message (pos_velox)
    pos_velox.x = pos.x
    pos_velox.y = pos.y
    pos_velox.velX = velocity.x
    pos_velox.velY = velocity.y

    # Publish the custom message
    pub.publish(pos_velox)

Buttons for Selecting Modes:

In [ ]:
send_goal_button = widgets.Button(value = False, description = "Target Position", disabled = False, button_style = '')

cancel_goal_button = widgets.Button(value = False, description = "Cancel Position", disabled = False, button_style = '')

Implement the numeric widgets `x` e `y`:

In [ ]:
x = widgets.BoundedFloatText(value = 0.0, description = 'x', min = -9.0, max = 9.0, 
                            style = {'description_width': 'initial'}, step = 0.1, 
                            layout = widgets.Layout(width = '100px'))
y = widgets.BoundedFloatText(value = 0.0, description = 'y', min = -9.0, max = 9.0,
                            style = {'description_width': 'initial'}, step = 0.1, 
                            layout = widgets.Layout(width = '100px'))

Then we have to create the callback functions related to the buttons:

In [ ]:
def clb_send(msg):
    global cancel_goal, goal_service
    goal_service = assignment_2_2022.msg.PlanningGoal()
    goal_service.target_pose.pose.position.x = x.value
    goal_service.target_pose.pose.position.y = y.value
    cancel_goal = False
    client.send_goal(goal_service)
    
def clb_cancel(msg):
    global cancel_goal, goal_service
    goal_service = None
    cancel_goal = True
    client.cancel_goal()

These two lines of code connect the button widgets to their respective callback functions.

In [ ]:
send_goal_button.on_click(clb_send)
cancel_goal_button.on_click(clb_cancel)

Create the function and the widget for the `PositionVisualizer`:

In [ ]:
class PositionVisualizer:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        # Robot Position Plot
        self.ln, = plt.plot([], [], 'ro', label = 'Robot Position')
        # Target's Position Plot
        self.goal_ln, = plt.plot([], [], 'b*', markersize = 10, label = 'Robot Goal Position')
        # Robot's Position Data Arrays
        self.x_data, self.y_data = [], []
        
    def plot_init(self):
        # Set Plot Title
        self.ax.set_title("Robot Odometry", fontsize = 20, fontweight = 'bold')
        # Set Plot Axis Labels
        self.ax.set_xlabel("X [m]", fontsize = 10, fontweight = "bold")
        self.ax.set_ylabel("Y [m]", fontsize = 10, fontweight = "bold")
        # Set Plot Axis Limits
        self.ax.set_xlim(-20, 20)
        self.ax.set_ylim(-20, 20)
        # Set Grid to True
        self.ax.grid(True)
        
        return self.ln, self.goal_ln
    
    def odom_callback(self, msg):
        # Callback Function used to update data
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)         
        
    def update_plot(self, frame):
        # Update Robot's Position on plot
        self.ln.set_data(self.x_data, self.y_data)  
        
        # If the goal is cancelled, remove the red marker
        if cancel_goal: 
            self.goal_ln.set_data([], [])
        # If the goal exists, set the red marker
        elif goal_service is not None:
            self.goal_ln.set_data(goal_service.target_pose.pose.position.x, goal_service.target_pose.pose.position.y)
        else:
            self.goal_ln.set_data([], [])
        
        return self.ln, self.goal_ln

Create the function and the widget for the `Goal_Visualizer`, for plotting the Reached/Cancelled Goals:

In [ ]:
class Goal_Visualizer:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        # Setting up the values
        self.reached = 0
        self.cancelled = 0
        # Set Plot Title
        self.ax.set_title("Reached/Cancelled Goals", fontsize = 20, fontweight = "bold")
        
        self.ax.grid(axis = 'y', color = 'grey', linestyle = '-', alpha = 0.5)
        # Set Plot Labels on x-axis
        self.labels = ('Reached', 'Cancelled')
        self.x_pos = np.arange(len(self.labels))
        self.ax.set_xticks(self.x_pos)
        self.ax.set_xticklabels(self.labels)
        # Set Plot Axis Limits
        self.ax.set_ylim([0, 10])
        self.ax.set_yticks(np.arange(0, 11, 1))
        self.bar_colors = ['green', 'red']
        self.bar_plot = self.ax.bar(self.x_pos, [self.reached, self.cancelled], align = 'center', color = self.bar_colors, width = 0.2)

    # Get the number of reached/cancelled goals
    def goal_callback(self, msg):
        global goals_cancelled, goals_reached
        self.reached += (msg.status.status == 3)
        self.cancelled += (msg.status.status == 2) 

    def update_plot(self, frame):
        
        for i, bar in enumerate(self.bar_plot):
            if i == 0:
                bar.set_height(self.reached)
            else:
                bar.set_height(self.cancelled)
            bar.set_color(self.bar_colors[i])
        return self.bar_plot

Create the function and the widget for the `Laser_Visualizer`:

In [ ]:
class Laser_Visualizer:
    def __init__(self):
        self.fig = plt.figure(figsize = (6,6))
        self.ax = plt.subplot(111, polar = True)
        self.ax.set_thetalim(-np.pi/2, np.pi/2)
        self.ax.set_theta_zero_location("N")
        self.laser = []
        self.angles = []
        self.ln, = self.ax.plot([], [], 'bo')
        
    def plot_init(self):
        # Set Plot Title and Plot Y Label
        self.ax.set_title("Robot Laser Scan", fontsize = 20, fontweight = 'bold')
        self.ax.set_ylabel("Wall Distance", fontsize = 10, fontweight = "bold")
        # Set the Grid
        self.ax.grid(True)
        return self.ln
    
    # Callback Function used to update data
    def laser_callback(self, msg):
        self.angles = list(np.arange(msg.angle_min, msg.angle_max + msg.angle_increment, msg.angle_increment))
        self.laser = list(msg.ranges)
        
    def update_plot(self, frame):
        self.ax.set_rmax(20)
        self.ln.set_data(self.angles, self.laser)  
        return self.ln

Now we initialize all the remaining part of code like node, subscriber, publisher and client:

In [ ]:
rospy.init_node('action_client_jupy')

pub = rospy.Publisher("/Position_velocity", Position_vel, queue_size = 1)

client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)

client.wait_for_server()

# Initialize the goals reached/cancelled
goals_cancelled = 0
goals_reached = 0

For last we visualize all the widged we create before:

In [ ]:
# Set the target position and send/cancell the goal position:

display(widgets.HBox([x, y]))
display(widgets.HBox([send_goal_button, cancel_goal_button]))

In [ ]:
# Plot the robot position:

position_visualizer = PositionVisualizer() 
sub = rospy.Subscriber('/odom', Odometry, position_visualizer.odom_callback)

animation_pos = FuncAnimation(position_visualizer.fig, position_visualizer.update_plot, init_func = position_visualizer.plot_init)
plt.show(block = True)

In [ ]:
# Plot the number of cancelled/reached goals:

visualize_goals = Goal_Visualizer()
results = rospy.Subscriber('/reaching_goal/result', assignment_2_2022.msg.PlanningActionResult, visualize_goals.goal_callback)

animation_goal = FuncAnimation(visualize_goals.fig, visualize_goals.update_plot, interval = 1000)
plt.show(block = True)

In [ ]:
# Plot the laser:

laser_visualizer = Laser_Visualizer()
sub = rospy.Subscriber('/scan', LaserScan, laser_visualizer.laser_callback)

animation_laser = FuncAnimation(laser_visualizer.fig, laser_visualizer.update_plot, init_func = laser_visualizer.plot_init)
plt.show(block = True)